In [1]:
import paddle
import paddle.nn as nn

from paddle.io import Dataset
from paddle.static import InputSpec

import paddle.fluid as fluid

from paddle.fluid.dygraph import Linear
from paddle.fluid.dygraph import Layer, to_variable
import paddle.fluid.dygraph as dygraph
import gc
import numpy as np
import pandas as pd
import os
import argparse
import random

/Users/chenguibin/anaconda3/lib/python3.7/site-packages/socks.py:58: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Callable


In [2]:
class Scaler(object):
    """
    Desc: Normalization utilities
    """
    def __init__(self):
        self.mean = 0.
        self.std = 1.

    def fit(self, data):
        # type: (paddle.tensor) -> None
        """
        Desc:
            Fit the data
        Args:
            data:
        Returns:
            None
        """
        self.mean = np.mean(data)
        self.std = np.std(data)

    def transform(self, data):
        # type: (paddle.tensor) -> paddle.tensor
        """
        Desc:
            Transform the data
        Args:
            data:
        Returns:
            The transformed data
        """
        mean = paddle.to_tensor(self.mean).type_as(data).to(data.device) if paddle.is_tensor(data) else self.mean
        std = paddle.to_tensor(self.std).type_as(data).to(data.device) if paddle.is_tensor(data) else self.std
        return (data - mean) / std

    def inverse_transform(self, data):
        # type: (paddle.tensor) -> paddle.tensor
        """
        Desc:
            Restore to the original data
        Args:
            data: the transformed data
        Returns:
            The original data
        """
        mean = paddle.to_tensor(self.mean) if paddle.is_tensor(data) else self.mean
        std = paddle.to_tensor(self.std) if paddle.is_tensor(data) else self.std
        return (data * std) + mean


In [3]:
class WindTurbineDataset(Dataset):
    """
    Desc: Data preprocessing,
          Here, e.g.    15 days for training,
                        3 days for validation,
                        and 6 days for testing
    """
    def __init__(self, data_path,
                 filename='my.csv',
                 flag='train',
                 size=None,
                 turbine_id=0,
                 task='MS',
                 target='Target',
                 scale=True,
                 start_col=2,       # the start column index of the data one aims to utilize
                 day_len=24 * 6,
                 train_days=15,     # 15 days
                 val_days=3,        # 3 days
                 test_days=6,       # 6 days
                 total_days=30      # 30 days
                 ):
        super().__init__()  
        self.unit_size = day_len    
        if size is None:
            self.input_len = self.unit_size   
            self.output_len = self.unit_size    
        else:
            self.input_len = size[0]
            self.output_len = size[1]
        # initialization
        assert flag in ['train', 'test', 'val']
        type_map = {'train': 0, 'val': 1, 'test': 2}
        self.set_type = type_map[flag]
        self.task = task
        self.target = target
        self.scale = scale
        self.start_col = start_col
        self.data_path = data_path
        self.filename = filename
        self.tid = turbine_id

        # If needed, we employ the predefined total_size (e.g. one month)
        self.total_size = self.unit_size * total_days   
        #
        self.train_size = train_days * self.unit_size  
        self.val_size = val_days * self.unit_size
        self.test_size = test_days * self.unit_size
        # self.test_size = self.total_size - train_size - val_size
        #
        # Or, if total_size is unavailable:
        # self.total_size = self.train_size + self.val_size + self.test_size
        self.__read_data__()

    def __read_data__(self):
        self.scaler = Scaler()
        df_raw = pd.read_csv(os.path.join(self.data_path, self.filename))
        border1s = [self.tid * self.total_size,  
                    self.tid * self.total_size + self.train_size - self.input_len,
                    self.tid * self.total_size + self.train_size + self.val_size - self.input_len
                    ]
        border2s = [self.tid * self.total_size + self.train_size,
                    self.tid * self.total_size + self.train_size + self.val_size,
                    self.tid * self.total_size + self.train_size + self.val_size + self.test_size
                    ]
        border1 = border1s[self.set_type]
        border2 = border2s[self.set_type]

        df_data = df_raw
        if self.task == 'M':
            cols_data = df_raw.columns[self.start_col:]
            df_data = df_raw[cols_data]
        elif self.task == 'MS':
            cols_data = df_raw.columns[self.start_col:]   
            df_data = df_raw[cols_data]
        elif self.task == 'S':
            df_data = df_raw[[self.tid, self.target]]

        # Turn off the SettingWithCopyWarning
        pd.set_option('mode.chained_assignment', None)
        df_data.replace(to_replace=np.nan, value=0, inplace=True)

        if self.scale:
            train_data = df_data[border1s[0]:border2s[0]]
            self.scaler.fit(train_data.values)
            data = self.scaler.transform(df_data.values)
        else:
            data = df_data.values
        self.data_x = data[border1:border2]
        self.data_y = data[border1:border2]
        self.raw_data = df_data[border1 + self.input_len:border2]  
    def get_raw_data(self):
        return self.raw_data

    def __getitem__(self, index):   
        # When sliding window not used, e.g. prediction without overlapped input/output sequences
        if self.set_type >= 3:
            index = index * self.output_len
        #
        # Standard use goes here.
        # Sliding window with the size of input_len + output_len
        s_begin = index
        s_end = s_begin + self.input_len
        r_begin = s_end
        r_end = r_begin + self.output_len
        seq_x = self.data_x[s_begin:s_end]   
        seq_y = self.data_y[r_begin:r_end] 
        return seq_x, seq_y

    def __len__(self):
        # In our case, the sliding window is adopted, the number of samples is calculated as follows
        if self.set_type < 3:
            return len(self.data_x) - self.input_len - self.output_len + 1
        # Otherwise, if sliding window is not adopted
        return int((len(self.data_x) - self.input_len) / self.output_len)

    def inverse_transform(self, data):
        return self.scaler.inverse_transform(data)


In [4]:
def prep_env():
    # type: () -> dict
    """
    Desc:
        Prepare the experimental settings
    Returns:
        The initialized arguments
    """
    parser = argparse.ArgumentParser(description='Long Term Wind Power Forecasting')
    ###
    parser.add_argument('--data_path', type=str, default='./data/', help='Path to the data file')
    parser.add_argument('--filename', type=str, default='wtbdata_245days.csv',
                        help='Filename of the input data, change it if necessary')
    parser.add_argument('--task', type=str, default='MS', help='The type of forecasting task, '
                                                               'options:[M, S, MS]; '
                                                               'M: multivariate --> multivariate, '
                                                               'S: univariate --> univariate, '
                                                               'MS: multivariate --> univariate')
    parser.add_argument('--target', type=str, default='Patv', help='Target variable in S or MS task')
    parser.add_argument('--checkpoints', type=str, default='./model_save/', help='Location of model checkpoints')
    parser.add_argument('--input_len', type=int, default=24, help='Length of the input sequence')
    parser.add_argument('--output_len', type=int, default=24, help='The length of predicted sequence')
    parser.add_argument('--start_col', type=int, default=3, help='Index of the start column of the meaningful variables')
    parser.add_argument('--in_var', type=int, default=10, help='Number of the input variables')
    parser.add_argument('--out_var', type=int, default=1, help='Number of the output variables')
    parser.add_argument('--day_len', type=int, default=144, help='Number of observations in one day')
    parser.add_argument('--train_size', type=int, default=100, help='Number of days for training')
    parser.add_argument('--val_size', type=int, default=100, help='Number of days for validation')
    parser.add_argument('--test_size', type=int, default=45, help='Number of days for testing')
    parser.add_argument('--total_size', type=int, default=100, help='Number of days for the whole dataset')
    parser.add_argument('--lstm_layer', type=int, default=2, help='Number of LSTM layers')
    parser.add_argument('--dropout', type=float, default=0.1, help='Dropout')
    parser.add_argument('--num_workers', type=int, default=5, help='#workers for data loader')
    parser.add_argument('--train_epochs', type=int, default=20, help='Train epochs')
    parser.add_argument('--batch_size', type=int, default=32, help='Batch size for the input training data')
    parser.add_argument('--patience', type=int, default=20, help='Early stopping patience')
    parser.add_argument('--lr', type=float, default=1e-4, help='Optimizer learning rate')
    parser.add_argument('--lr_adjust', type=str, default='type1', help='Adjust learning rate')
    parser.add_argument('--model_selection', type=str, default='LSTM', help='Model selection:LSTM,En_DecoderGRE,Atten_EnDecoder_GRU')
    parser.add_argument('--use_gpu', type=bool, default=True, help='Whether or not use GPU')
    parser.add_argument('--gpu', type=int, default=0, help='GPU ID')
    # parser.add_argument('--use_multi_gpu', action='store_true', default=False, help='Use multiple gpus or not')
    parser.add_argument('--capacity', type=int, default=1, help="The capacity of a wind farm, "
                                                                  "i.e. the number of wind turbines in a wind farm")
    parser.add_argument('--turbine_id', type=int, default=0, help='Turbine ID')
    parser.add_argument('--pred_file', type=str, default='./predict.py',
                        help='The path to the script for making predictions')
    parser.add_argument('--stride', type=int, default=1, help='The stride that a window adopts to roll the test set')
    parser.add_argument('--is_debug', type=bool, default=False, help='True or False')
    #args = parser.parse_args()
    args = parser.parse_args(args=[])
    settings = {
        "data_path": args.data_path,
        "filename": args.filename,
        "task": args.task,
        "target": args.target,
        "checkpoints": args.checkpoints,
        "input_len": args.input_len,
        "output_len": args.output_len,
        "start_col": args.start_col,
        "in_var": args.in_var,
        "out_var": args.out_var,
        "day_len": args.day_len,
        "train_size": args.train_size,
        "val_size": args.val_size,
        "test_size": args.test_size,
        "total_size": args.total_size,
        "lstm_layer": args.lstm_layer,
        "dropout": args.dropout,
        "num_workers": args.num_workers,
        "train_epochs": args.train_epochs,
        "batch_size": args.batch_size,
        "patience": args.patience,
        "lr": args.lr,
        "lr_adjust": args.lr_adjust,
        "model_selection": args.model_selection,
        "capacity": args.capacity,
        "turbine_id": args.turbine_id,
        "pred_file": args.pred_file,
        "stride": args.stride,
        "is_debug": args.is_debug
    }
    ###
    # Prepare the GPUs
    if paddle.device.is_compiled_with_cuda():
        args.use_gpu = True
        paddle.device.set_device('gpu:{}'.format(args.gpu))
    else:
        args.use_gpu = False
        paddle.device.set_device('cpu')
    print("Experimental settings are: \n{}".format(str(settings)))
    return settings


In [5]:
settings = prep_env()

Experimental settings are: 
{'data_path': './data/', 'filename': 'wtbdata_245days.csv', 'task': 'MS', 'target': 'Patv', 'checkpoints': './model_save/', 'input_len': 24, 'output_len': 24, 'start_col': 3, 'in_var': 10, 'out_var': 1, 'day_len': 144, 'train_size': 100, 'val_size': 100, 'test_size': 45, 'total_size': 100, 'lstm_layer': 2, 'dropout': 0.1, 'num_workers': 5, 'train_epochs': 20, 'batch_size': 32, 'patience': 20, 'lr': 0.0001, 'lr_adjust': 'type1', 'model_selection': 'LSTM', 'capacity': 1, 'turbine_id': 0, 'pred_file': './predict.py', 'stride': 1, 'is_debug': False}


In [6]:
from typing import Callable
import time
import numpy as np
import paddle
import paddle.nn as nn
import random
from paddle.io import DataLoader
from model import BaselineRNNModel

def adjust_learning_rate(optimizer, epoch, args):
    # type: (paddle.optimizer.Adam, int, dict) -> None
    """
    Desc:
        Adjust learning rate
    Args:
        optimizer:
        epoch:
        args:
    Returns:
        None
    """
    # lr = args.lr * (0.2 ** (epoch // 2))
    lr_adjust = {}
    if args["lr_adjust"] == 'type1':
        # learning_rate = 0.5^{epoch-1}
        lr_adjust = {epoch: args["lr"] * (0.50 ** (epoch - 1))}
    elif args["lr_adjust"] == 'type2':
        lr_adjust = {
            2: 5e-5, 4: 1e-5, 6: 5e-6, 8: 1e-6,
            10: 5e-7, 15: 1e-7, 20: 5e-8
        }
    if epoch in lr_adjust:
        lr = lr_adjust[epoch]
        optimizer.set_lr(lr)


In [7]:
class Experiment(object):
    """
    Desc:
        The experiment to train, validate and test a model
    """
    def __init__(self, args):
        # type: (dict) -> None
        """
        Desc:
            __init__
        Args:
            args: the arguments to initialize the experimental environment
        """
        self.model = BaselineRNNModel(args)
        self.args = args

    def get_model(self):
        # type: () -> BaselineGruModel
        """
        Desc:
            the model
        Returns:
            An instance of the model
        """
        return self.model

    def get_args(self):
        # type: () -> dict
        """
        Desc:
            Get the arguments
        Returns:
            A dict
        """
        return self.args

    def get_data(self, flag):
        # type: (str) -> (WindTurbineDataset, DataLoader)
        """
        Desc:
            get_data
        Args:
            flag: train or test
        Returns:
            A dataset and a dataloader
        """
        if flag == 'test':
            shuffle_flag = False
            drop_last = True
        else:
            shuffle_flag = True
            drop_last = True
        data_set = WindTurbineDataset(
            data_path=self.args["data_path"],
            filename=self.args["filename"],
            flag=flag,
            size=[self.args["input_len"], self.args["output_len"]],
            task=self.args["task"],
            target=self.args["target"],
            start_col=self.args["start_col"],
            turbine_id=self.args["turbine_id"],
            day_len=self.args["day_len"],
            train_days=self.args["train_size"],
            val_days=self.args["val_size"],
            test_days=self.args["test_size"],
            total_days=self.args["total_size"]
        )
        data_loader = DataLoader(
            data_set,
            batch_size=self.args["batch_size"],
            shuffle=shuffle_flag,
            num_workers=self.args["num_workers"],
            drop_last=drop_last
        )
        return data_set, data_loader

    def get_optimizer(self):
        # type: () -> paddle.optimizer.Adam
        """
        Desc:
            Get the optimizer
        Returns:
            An optimizer
        """
        clip = paddle.nn.ClipGradByNorm(clip_norm=50.0)
        model_optim = paddle.optimizer.Adam(parameters=self.model.parameters(),
                                            learning_rate=self.args["lr"],
                                            grad_clip=clip)
        return model_optim

    @staticmethod
    def get_criterion():
        # type: () -> nn.MSELoss
        """
        Desc:
            Use the mse loss as the criterion
        Returns:
            MSE loss
        """
        criterion = nn.MSELoss(reduction='mean')  
        return criterion

    def process_one_batch(self, batch_x, batch_y):
        # type: (paddle.tensor, paddle.tensor) -> (paddle.tensor, paddle.tensor)
        """
        Desc:
            Process a batch
        Args:
            batch_x:
            batch_y:
        Returns:
            prediction and ground truth
        """
        batch_x = batch_x.astype('float32')
        batch_y = batch_y.astype('float32')
        sample = self.model(batch_x)
        #
        # If the task is the multivariate-to-univariate forecasting task,
        # the last column is the target variable to be predicted
        f_dim = -1 if self.args["task"] == 'MS' else 0
        #
        batch_y = batch_y[:, -self.args["output_len"]:, f_dim:].astype('float32')
        sample = sample[..., :, f_dim:].astype('float32')
        return sample, batch_y


fix_seed = 3407
random.seed(fix_seed)
paddle.seed(fix_seed)
np.random.seed(fix_seed)


def traverse_wind_farm(method, params, model_path, flag='train'):
    # type: (Callable, dict, str, str) -> list
    """
    Desc:
        Traverse the turbines in a wind farm on by one
    Args:
        method: the method for training or testing on the records of one turbine
        params: the arguments initialized
        model_path: the folder name of the model
        flag: 'train' or 'test'
    Returns:
        Predictions (for test) or None
    """
    responses = []
    start_time = time.time()
    for i in range(params["capacity"]):
        params["turbine_id"] = i
        exp = Experiment(params)
        if 'train' == flag:
            print('>>>>>>> Training Turbine {:3d} >>>>>>>>>>>>>>>>>>>>>>>>>>\n'.format(i))
            method(exp, model_path, is_debug=params["is_debug"])
        elif 'test' == flag:
            print('>>>>>>> Forecasting Turbine {:3d} >>>>>>>>>>>>>>>>>>>>>>>>>>\n'.format(i))
            res = method(exp, model_path)
            responses.append(res)
        else:
            pass
        paddle.device.cuda.empty_cache()
        if params["is_debug"]:
            end_time = time.time()
            print("Elapsed time for {} turbine {} is {} secs".format("training" if "train" == flag else "predicting", i,
                                                                     end_time - start_time))
            start_time = end_time
    if 'test' == flag:
        return responses


In [8]:
experiment = Experiment(settings)
args = experiment.get_args()
model = experiment.get_model()
train_data,tran_loader= experiment.get_data(flag='train')

/Users/chenguibin/anaconda3/lib/python3.7/site-packages/paddle/fluid/reader.py:356: UserWarning: DataLoader with multi-process mode is not supported on MacOs and Windows currently. Please use signle-process mode with num_workers = 0 instead
  "DataLoader with multi-process mode is not supported on MacOs and Windows currently." \


In [9]:
class EarlyStopping(object):
    """
    Desc:
        EarlyStopping
    """
    def __init__(self, patience=30, verbose=False, delta=0):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.best_model = False

    def save_checkpoint(self, val_loss, model, path, tid):
        # type: (nn.MSELoss, BaselineGruModel, str, int) -> None
        """
        Desc:
            Save current checkpoint
        Args:
            val_loss: the validation loss
            model: the model
            path: the path to be saved
            tid: turbine ID
        Returns:
            None
        """
        self.best_model = True
        self.val_loss_min = val_loss
        paddle.save(model.state_dict(), path + '/' + 'model_' + str(tid))  

    def __call__(self, val_loss, model, path, tid):
        # type: (nn.MSELoss, BaselineGruModel, str, int) -> None
        """
        Desc:
            __call__
        Args:
            val_loss: the validation loss
            model: the model
            path: the path to be saved
            tid: turbine ID
        Returns:
            None
        """
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model, path, tid)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.best_model = False
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.update_hidden = True
            self.save_checkpoint(val_loss, model, path, tid)
            self.counter = 0


In [10]:
def val(experiment, data_loader, criterion):
    # type: (Experiment, DataLoader, Callable) -> np.array
    """
    Desc:
        Validation function
    Args:
        experiment:
        data_loader:
        criterion:
    Returns:
        The validation loss
    """
    validation_loss = []
    for i, (batch_x, batch_y) in enumerate(data_loader):
        sample, true = experiment.process_one_batch(batch_x, batch_y)
        loss = criterion(sample, true)
        validation_loss.append(loss.item())
    validation_loss = np.average(validation_loss)
    return validation_loss



In [11]:
def train_and_val(experiment, model_folder, is_debug=False):
    # type: (Experiment, str, bool) -> None
    """
    Desc:
        Training and validation
    Args:
        experiment:
        model_folder: folder name of the model
        is_debug:
    Returns:
        None
    """
    args = experiment.get_args()
    model = experiment.get_model()
    train_data, train_loader = experiment.get_data(flag='train')
    val_data, val_loader = experiment.get_data(flag='val')

    path_to_model = os.path.join(args["checkpoints"], model_folder)
    if not os.path.exists(path_to_model):
        os.makedirs(path_to_model)

    time_now = time.time()
    early_stopping = EarlyStopping(patience=args["patience"], verbose=True)
    model_optim = experiment.get_optimizer()
    criterion = Experiment.get_criterion()

    epoch_start_time = time.time()
    for epoch in range(args["train_epochs"]):
        iter_count = 0
        train_loss = []
        model.train()
        for i, (batch_x, batch_y) in enumerate(train_loader):
            iter_count += 1
            sample, truth = experiment.process_one_batch(batch_x, batch_y)
            loss = criterion(sample, truth)
            train_loss.append(loss.item())
            loss.backward()
            model_optim.minimize(loss)
            model_optim.step()
        val_loss = val(experiment, val_loader, criterion)
        print('Epoch: {}, Train loss average'.format(epoch), np.mean(np.array(train_loss)))

        if is_debug:
            train_loss = np.average(train_loss)
            epoch_end_time = time.time()
            print("Epoch: {}, \nTrain Loss: {}, \nValidation Loss: {}".format(epoch, train_loss, val_loss))
            print("Elapsed time for epoch-{}: {}".format(epoch, epoch_end_time - epoch_start_time))
            epoch_start_time = epoch_end_time

        # Early Stopping if needed
        early_stopping(val_loss, model, path_to_model, args["turbine_id"])
        if early_stopping.early_stop:
            print("Early stopped! ")
            break
        adjust_learning_rate(model_optim, epoch + 1, args)


In [12]:
if __name__ == "__main__":
    settings = prep_env()
    #
    # Set up the initial environment
    # Current settings for the model
    cur_setup = '{}_t{}_i{}_o{}_ls{}_train{}_val{}_model{}'.format(
        settings["filename"], settings["task"], settings["input_len"],  settings["output_len"], settings["lstm_layer"],
        settings["train_size"], settings["val_size"], settings["model_selection"]
    )
    traverse_wind_farm(train_and_val, settings, cur_setup)

Experimental settings are: 
{'data_path': './data/', 'filename': 'wtbdata_245days.csv', 'task': 'MS', 'target': 'Patv', 'checkpoints': './model_save/', 'input_len': 24, 'output_len': 24, 'start_col': 3, 'in_var': 10, 'out_var': 1, 'day_len': 144, 'train_size': 100, 'val_size': 100, 'test_size': 45, 'total_size': 100, 'lstm_layer': 2, 'dropout': 0.1, 'num_workers': 5, 'train_epochs': 20, 'batch_size': 32, 'patience': 20, 'lr': 0.0001, 'lr_adjust': 'type1', 'model_selection': 'LSTM', 'capacity': 1, 'turbine_id': 0, 'pred_file': './predict.py', 'stride': 1, 'is_debug': False}
>>>>>>> Training Turbine   0 >>>>>>>>>>>>>>>>>>>>>>>>>>

Epoch: 0, Train loss average 4.226106554003699
Epoch: 1, Train loss average 2.84319256991148
Epoch: 2, Train loss average 2.7148902099579573
Epoch: 3, Train loss average 2.565620473985161
Epoch: 4, Train loss average 2.4215440449437926
Epoch: 5, Train loss average 2.370458953614746
Epoch: 6, Train loss average 2.3478238694369793
Epoch: 7, Train loss average 2.3

In [45]:
#record parameter number
model = experiment.get_model()
total_params = sum(p.numpy().size for p in model.parameters())
print("Total parameters in the model:", total_params)

Total parameters in the model: 22801


In [13]:
cur_setup = '{}_t{}_i{}_o{}_ls{}_train{}_val{}_model{}'.format(
        settings["filename"], settings["task"], settings["input_len"],  settings["output_len"], settings["lstm_layer"],
        settings["train_size"], settings["val_size"], settings["model_selection"]
    )
model_folder=cur_setup

In [24]:
test_data, test_loader = experiment.get_data(flag='test')
predictions = []
true_lst = []
for i in range(args["test_size"]):
    single_seqx, single_seqy = test_data.__getitem__(i*args["input_len"])
    #print(single_seqx[1].shape)
    single_seqx1 = single_seqx.astype('float32')
    #single_seqx2 = single_seqx[1].astype('float32')
    single_seqy = single_seqy.astype('float32')
    f_dim = -1 if args["task"] == 'MS' else 0
    
    single_seqy = single_seqy[-args["output_len"]:, f_dim:].astype('float32')
        
    enc_x1 = paddle.to_tensor(single_seqx1).unsqueeze(0)
    #enc_x2 = paddle.to_tensor(single_seqx2).unsqueeze(0)
    enc_y = paddle.to_tensor(single_seqy).unsqueeze(0)
    #print(enc_x.shape)
    model = experiment.get_model()
    
    path_to_model = os.path.join(args["checkpoints"], model_folder, 'model_{}'.format(0))
    print('path:',path_to_model)
    model.set_state_dict(paddle.load(path_to_model))
    print('encx1:',enc_x1)
    single_pred = experiment.model(enc_x1)
    predictions.append(np.array(single_pred))
    true_lst.append(np.array(single_seqy))

path: ./model_save/wtbdata_245days.csv_tMS_i24_o24_ls2_train100_val100_modelLSTM/model_0
encx1: Tensor(shape=[1, 24, 10], dtype=float32, place=CPUPlace, stop_gradient=True,
       [[[-0.28637528, -0.32571971, -0.27001080, -0.19650117,  0.57271689,
          -0.31318519, -0.31318519, -0.31318519, -0.31866905,  2.97942853],
         [-0.28855142, -0.31614473, -0.27075067, -0.19663173,  0.57271689,
          -0.31318519, -0.31318519, -0.31318519, -0.31871256,  2.41219878],
         [-0.29072756, -0.32010528, -0.27048954, -0.19684935,  0.57271689,
          -0.31318519, -0.31318519, -0.31318519, -0.31871256,  2.11567950],
         [-0.28994414, -0.33342320, -0.26517978, -0.19358514,  0.57271689,
          -0.31318519, -0.31318519, -0.31318519, -0.31875607,  2.29420900],
         [-0.28802913, -0.32806993, -0.26957557, -0.19454265,  0.57271689,
          -0.31318519, -0.31318519, -0.31318519, -0.31862551,  2.54228783],
         [-0.28554836, -0.32519743, -0.27092478, -0.19645764,  0.5727168

encx1: Tensor(shape=[1, 24, 10], dtype=float32, place=CPUPlace, stop_gradient=True,
       [[[-0.28637528, -0.33233514, -0.32236847, -0.22605300, -0.20881805,
          -0.31749395, -0.31749395, -0.31749395, -0.23937090,  2.74915051],
         [-0.28990063, -0.31684110, -0.32450107, -0.22531311, -0.20881805,
          -0.31784213, -0.31784213, -0.31784213, -0.12064131,  2.20228934],
         [-0.29185912, -0.30952930, -0.32123688, -0.22522606, -0.20881805,
          -0.31980065, -0.31980065, -0.31980065, -0.13639648,  2.07141685],
         [-0.29473162, -0.30182579, -0.31684110, -0.21991631, -0.19293231,
          -0.32184619, -0.32184619, -0.32184619, -0.17700304,  1.48812735],
         [-0.29516685, -0.31196657, -0.31993121, -0.22322403, -0.17121455,
          -0.32184619, -0.32184619, -0.32184619, -0.43356866,  1.42584646],
         [-0.29503629, -0.31745040, -0.31379452, -0.21756609, -0.17121455,
          -0.32184619, -0.32184619, -0.32184619, -0.52278996,  1.61029518],
         [

In [25]:
predictions = np.array(predictions)

In [26]:
predictions = predictions.reshape(-1, predictions.shape[-2], predictions.shape[-1])
predictions.shape

(45, 24, 1)

In [16]:
predictions = predictions[:,:,-1]

In [27]:
preds = []
for l in predictions:
    preds.append(l.squeeze())
#preds

In [28]:
pp = []
for k in preds:
    pp.append(np.array(k.squeeze()))
#pp

In [29]:
pp[0].shape

(24,)

In [30]:
ppp = []
for w in range(len(pp)):
    for j in range(len(pp[w])):
        ppp.append(pp[w][j])
#ppp

In [31]:
true_lst = np.array(true_lst)
true_lst.shape

(45, 24, 1)

In [32]:
true_lst = true_lst.reshape(-1, true_lst.shape[-2], true_lst.shape[-1])
true_lst.shape

(45, 24, 1)

In [33]:
true_lst = true_lst[:,:,-1]

In [34]:
trs = []
for l in true_lst:
    trs.append(l.squeeze())
#trs

In [35]:
tt = []
for k in trs:
    tt.append(np.array(k.squeeze()))
#tt

In [36]:
ttt = []
for w in range(len(tt)):
    for j in range(len(tt[w])):
        ttt.append(tt[w][j])
#ttt

In [37]:
eva_preds = np.array(ppp)
ground_tr = np.array(ttt)

In [38]:
eva_preds.shape

(1080,)

In [39]:
ground_tr.shape

(1080,)

In [40]:
ground_tr

array([ 0.9225514 ,  1.2209423 ,  1.2112802 , ..., -0.31884313,
       -0.31884313, -0.31884313], dtype=float32)

In [ ]:
all_data = np.transpose(np.vstack((eva_preds, ground_tr)))
all_data.shape

In [47]:

# Create a DataFrame with column name "pred"
df = pd.DataFrame(eva_preds, columns=['RNNpred'])

# Save the DataFrame to a CSV file
df.to_csv('data/RNN24steps_preds1080.csv', index=False)